In [1]:
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path
from typing import List

import utm
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry as sg
import shapely.ops as so

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
def embed_map(m):
    """
    Workaround taken from https://github.com/python-visualization/folium/issues/812
    for displaying Folium maps with lots of features in Chrome-based browsers.
    """
    from IPython.display import IFrame

    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

In [3]:
path = DATA_DIR/'cairns_gtfs.zip'
gk.list_feed(path)



,file_name,file_size
0,calendar_dates.txt,387
1,routes.txt,1478
2,trips.txt,143081
3,stops.txt,26183
4,agency.txt,199
5,stop_times.txt,2561019
6,shapes.txt,864694
7,calendar.txt,337


In [4]:
feed = (
    gk.read_feed(path, dist_units='km')
    .append_dist_to_stop_times()
)
feed.describe()

/home/araichev/.virtualenvs/gtfs_kit/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/home/araichev/.virtualenvs/gtfs_kit/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


,indicator,value
0,agencies,[Department of Transport and Main Roads - Tran...
1,timezone,Australia/Brisbane
2,start_date,20140526
3,end_date,20141228
4,num_routes,22
5,num_trips,1339
6,num_stops,416
7,num_shapes,54
8,sample_date,20140529
9,num_routes_active_on_sample_date,20


In [5]:
url = "http://transitfeeds.com/p/10-15-transit/936/latest/download"
gk.read_feed(url, dist_units="km").describe()

,indicator,value
0,agencies,[10-15 Transit]
1,timezone,America/Chicago
2,start_date,20171201
3,end_date,20210101
4,num_routes,1
5,num_trips,8
6,num_stops,66
7,num_shapes,1
8,sample_date,20171207
9,num_routes_active_on_sample_date,0


In [ ]:
collection = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [
            145.7731032371521,
            -16.917382184901566
          ],
          [
            145.7735753059387,
            -16.917053719034044
          ]
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [
            145.76706290245056,
            -16.916376256373532
          ],
          [
            145.76752424240112,
            -16.915986201009297
          ]
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [
            145.75278282165527,
            -16.912598844152377
          ],
          [
            145.7535982131958,
            -16.914076970997165
          ]
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [
            145.7585334777832,
            -16.903072859722965
          ],
          [
            145.75737476348877,
            -16.90405832871205
          ]
        ]
      }
    }
  ]
}
screen_lines = gpd.GeoDataFrame.from_features(collection["features"], crs=gk.WGS84)
display(screen_lines)

dates = feed.get_first_week()[:1]
%time f1 = bingo(feed, screen_lines.iloc[[0]], dates)
print(type(f1))
display(f1)

In [ ]:
%%time

for __ in range(screen_lines.shape[0]):
    f2 = feed.compute_screen_line_counts(screen_lines.geometry.iat[0], dates)

In [ ]:
g1 = f1.loc[lambda x: x.screen_line_id == "sl0"].sort_values("trip_id")
g2 = f2.sort_values("trip_id")

In [ ]:
(
    g1.crossing_time.equals(g2.crossing_time),
    g1.crossing_direction.equals(g2.orientation)
)

In [ ]:
embed_map(feed.map_routes(feed.routes.route_id.iloc[:4], include_stops=False))
